# Etapa de Pré-Processamento

O ideal é que esta etapa seja realizada enquanto os dados estejam crus (antes de transformar em um objeto MNE ou transformado em um `RawData`). Esta afirmação ocorre pois os dados transformados em um objeto MNE `Epoch` ou `Evoked` não possuem todo os métodos para aplicar os **filtros temporais**, por exemplo, a ausência da negação de frequência (*notch_filter*).

Contudo, podemos obter bons resultados com filtros passa-baixa (*low pass*) ou passa-alta (*high pass*), configurando valores abaixo e acima da frequência de corte desejada respectivamente.

In [1]:
import numpy as np
import json
import mne
import scipy
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from scipy.signal import stft
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

%matplotlib inline

In [2]:
# carregamento do dataset (FIF file)
epochs = mne.read_epochs('files/ssvep-epo.fif')
print(epochs)

# filtranndo apenas alguns canais
epochs.pick_channels(['E101', 'E118', 'E119'])
print(epochs)

# configurando um evento para apresentação dos resultados
event = '4'

Reading files/ssvep-epo.fif ...


FileNotFoundError: [Errno 2] No such file or directory: 'files/ssvep-epo.fif'

In [ ]:
# apresentando o gráfico de um único evento com todos os eletrodos escolhidos SEM FILTRO
matplotlib.rcParams['figure.figsize'] = [12., 8.]
epochs[event].plot_psd(fmin = 5, fmax = 14)
print('')

## Filtros temporais

Apesar do nome ocorrer como "temporal", o resultado destes filtros ficam melhor visíveis em um gráfico do domínio da frequência, ao invés do domínio do tempo como o nome sugere. O filtro é dito temporal pois influencia diretamente na **periodicidade** das diversas frequências presentes em um sinal de EEG por exemplo. Mesmo que singelo, é possível perceber que o gráfico no domínio do tempo é mais suave de acordo com a quantidade de frequência "excluída" do sinal. Porém, vemos a diferença direta no domínio da frequência.

Existem 4 tipos de filtros temporais a saber:
- passa-alta (*high pass*)
- passa-baixa (*low pass*)
- passa-faixa (*band pass*)
- rejeita-faixa (*notch*)

Mostraremos o resultado de cada um deles (e o *notch filter* quebrando em `Raw`)

### *High pass filter*

In [ ]:
epo_h = epochs.copy().filter(l_freq=None, h_freq=14.)
epo_h[event].plot_psd(fmin = 5, fmax = 14)
print('')

Onde ocorre a "quebra" do sinal ou aplicação do filtro, é mostrado uma linha pontilhada, indicando para o *high-pass* que a partir daquele ponto (14 Hz) a frequência foi cortada. 

### *Low pass filter*

In [ ]:
epo_l = epochs.copy().filter(l_freq=5, h_freq=None)
epo_l[event].plot_psd(fmin = 5, fmax = 14)
print('')

Aqui percebemos o corte das frequências abaixo de 5 Hz e o restante do sinal igual ao original.

### *Band pass filter*

In [ ]:
epo_b = epochs.copy().filter(l_freq=5, h_freq=14)
for i in range (9):
    epo_b = epo_b.copy().filter(l_freq=5, h_freq=14)

epo_b[event].plot_psd(fmin = 5, fmax = 14)
print('')

Neste exemplo vimos que é possível realizar o corte das frequências acima e abaixo do desejado. Porém, o resultado do que foi cortado acima e abaixo das frequências não foi igual ao resultado apresentado no corte das frequências originais. Iremos analisar isso com mais detalhes aplicando *low* e *high filter* em sequência e comparar com o resultado gerado anteriormente, obtendo algo bem parecido.

In [ ]:
epo_b2 = epochs.copy().filter(l_freq=5, h_freq=None)
epo_b2.filter(l_freq=None, h_freq=14)
epo_b2[event].plot_psd(fmin = 5, fmax = 14)
print('')

Por fim, o *notch*, após reduzir o objeto para um `Raw` para simples visualização.

### notch filter

In [ ]:
a = epochs.get_data()
print(a.shape)
a = a.transpose(1, 0, 2)
a = a.reshape(3, 125 * 1205)

# criando o objeto `info` (o restante dos valores já temos)
info = mne.create_info(ch_names=3,
                       sfreq=241.,
                       ch_types='eeg')

raw = mne.io.RawArray(a, info)

# raw.notch_filter?

### Aplicações multiplas de filtros causam diferença?

Vamos analisar o que acontece se atenuarmos diversas vezes o mesmo filtro.

In [ ]:
# aplicar os filtros N vezes necessárias e verificar se há modificações

# realizar ZOOM nas abcissas (frequência) para ver os sinais filtrados "mais de perto"
print('')

## Filtros espaciais

Filtros espaciais são utilizados para escolher canais de dados que melhor respondem a uma determinada tarefa. No caso do paradigma SSVEP, espera-se encontrar o conjunto de eletrodos que possuem respostas mais significativas em relação à frequências evocadas. Um algoritmo muito utilizado para este fim é o CAR (*Common Average Reference*).

Uma filtragem espacial pelo método CAR subtrai os artefatos comuns a todos os eletrodos. Comumente em BCI, para a redução de sinais não desejados, são posicionados eletrodos de referência em locais que tentam subtrair os potenciais comuns em todos os outros eletrodos. A técnica de filtragem espacial CAR reduz as componentes que estão em maior número nos eletrodos, subtraindo cada amostra do sinal pelo valor médio da diferença de potencial calculada para todos os eletrodos e a diferença aferida em cada eletrodo.

O método CAR é definito como:

\begin{equation}
V_i^{CAR} = V_i^{ER} - \frac{1}{N_e} \sum\limits_{j=1}^{N_e} V_j^{ER}
\end{equation}

onde $V_i^{ER}$ é o potencial do i-ésimo eletrodo e ${N_e}$ é o número de eletrodos utilizados.

No MNE a implementação do CAR se dá pela aplicação do método `set_eeg.reference`:

In [ ]:
##Plot do epochs para referência
epochs[event].plot_psd()

#epo_ref = mne.set_eeg_reference(epochs)




#Plot do epoch após passar pelo filtro de referência
#epo_ref[0][event].plot_psd()

In [ ]:
data = np.load('files/data.npy')
labels = np.load('files/labels.npy')

desc_file = open('files/descriptor.json')
deor = json.loads(desc_file.read())
desc_file.close()

print('Estruturas => dados', data.shape, 'labels', labels.shape)
print(labels)

data = data[:, :256, :]

trial_duration = 5
sampling_frequency = data.shape[-1] / trial_duration
montage = mne.channels.make_standard_montage('EGI_256')
ch_names = data.shape[1]
ch_types = 'eeg'

# primeiramente devemos criar o objeto info
info = mne.create_info(montage.ch_names, sampling_frequency, ch_types)

# set experiments montage
info.set_montage(montage)

# por fim a criação do EpochsArray
events = np.array([[index, 0, event] for index, event in enumerate(labels)])
# objeto MNE epoch
#'E128', 'E129', 'E138', 'E139', 'E140', 'E141'
epoch = mne.EpochsArray(data, info, events)



channels = ['E101', 'E118', 'E119']



### Questões de projeto

1. Como saber quais são os eletrodos a serem utilizados como referência?
2. A média de TODOS os eletrodos é um bom chute?
3. Como utilizar as informações de retorno do método `set_eeg_reference`

1. De acordo com https://www.educamaisbrasil.com.br/enem/biologia/cerebro, a região do cérebro que processa informações visuais é o Lobo Occipital ou a região traseira-inferior do crânio. Sendo assim, como o experimento trata de elementos visuais, os eletrodos a serem utilizados são os occipitais e para-occipitais.

2. Não, pois entrariam para o cálculo, sinais de tato, olfato, paladar e outras sensações, gerando um ruído para o cálculo de média. O correto, para este experimento, seria realizar uma média de todos eletrodos occipitais e para-occipitais, ou poderia ser aplicado algum outro filtro.

3. As informações de retorno do `set_eeg_reference` especificam explicitamente uma referência de eletrodos, no caso do experimento, os eletrodos principais referentes à visão, são os 'E116' = O1, 'E126' = Oz e 'E150' = O2. Assim filtrando menores ruídos de outros eletrodos. Porém, ainda é necessário aplicar um filtro para amenizar as frequências não desejadas.